# Model Inference

Inference using `pyhf` Python API

In [ ]:
import pyhf
import json

In [ ]:
with open("data/2-bin_1-channel.json") as serialized:
  spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
model = workspace.model(poi_name="mu")

In [ ]:
pars = model.config.suggested_init()
data = workspace.data(model)

Creating the log-likelihood conditioned on the data
$$
L(\vec{\theta}|\vec{x}) = k \cdot p(\vec{x} | \vec{\theta})
$$

In [ ]:
model.logpdf(pars, data)

moar inference

In [ ]:
bestfit_pars, twice_nll = pyhf.infer.mle.fit(data, model, return_fitted_val=True)
print(bestfit_pars)

objective function is twice the negative log-likelihood

In [ ]:
-2 * model.logpdf(bestfit_pars, data) == twice_nll

## Hypothesis Testing

In [ ]:
cls_obs, cls_exp = pyhf.infer.hypotest(1.0, data, model, return_expected_set=True)
cls_obs, cls_exp

In [ ]:
import numpy as np

results = []
poi_vals = np.linspace(0,5,41)
for mu in poi_vals:
    results.append(
        pyhf.infer.hypotest(mu, data, model, return_expected_set=True)
    )

In [ ]:
obs = [rr[0][0] for rr in results]
exp = [rr[1][2][0] for rr in results]

print('Upper Limit (obs): µ = {:0.3}'.format(np.interp(0.05,obs[::-1],poi_vals[::-1])))
print('Upper Limit (exp): µ = {:0.3}'.format(np.interp(0.05,exp[::-1],poi_vals[::-1])))

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import pyhf.contrib.viz.brazil

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_size_inches(7, 5)

ax.set_xlabel(r"$\mu$ (POI)")
ax.set_ylabel(r"$\mathrm{CL}_{s}$")

pyhf.contrib.viz.brazil.plot_results(ax, poi_vals, results)